In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install "flair" -q
!pip install "scispacy" -q
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz -q

LSTM-CRF : [link](https://github.com/flairNLP/flair/blob/master/resources/docs/HUNFLAIR_TUTORIAL_2_TRAINING.md)

Transformer: [link](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_TRAINING_SEQUENCE_LABELER.md)

In [3]:
import flair
from flair.data import Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import (
    WordEmbeddings, FlairEmbeddings, StackedEmbeddings
)
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
flair.__version__

'0.12.2'

In [4]:
DATA_PATH = "/content/drive/Shareddrives/CIS522-Project/data"
MODEL_PATH = "/content/drive/Shareddrives/CIS522-Project/models"

In [5]:
columns = {0:"text", 1:"ner"}

filename = "flair_ner_train_augmented.txt"
test_file = "flair_ner_test.txt"

corpus = ColumnCorpus(
    DATA_PATH, columns, train_file=filename, test_file=test_file
)
tag_dictionary = corpus.make_label_dictionary(label_type="ner", add_unk=False)
print(tag_dictionary.get_items())

2023-04-15 18:59:52,902 Reading data from /content/drive/Shareddrives/CIS522-Project/data
2023-04-15 18:59:52,907 Train: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_train_augmented.txt
2023-04-15 18:59:52,909 Dev: None
2023-04-15 18:59:52,911 Test: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_test.txt
2023-04-15 19:00:20,566 Computing label dictionary. Progress:


2776it [00:00, 33562.17it/s]

2023-04-15 19:00:20,709 Dictionary created for label 'ner' with 9 values: Drug (seen 5612 times), ADE (seen 4548 times), Reason (seen 1273 times), Strength (seen 962 times), Route (seen 823 times), Frequency (seen 725 times), Form (seen 630 times), Dosage (seen 607 times), Duration (seen 119 times)
['Drug', 'ADE', 'Reason', 'Strength', 'Route', 'Frequency', 'Form', 'Dosage', 'Duration']


In [6]:
# weight_dict = {
#     'Drug': 5502/5502,
#     'Strength': 5502/945,
#     'Form': 5502/613,
#     'Frequency': 5502/685,
#     'Route': 5502/789,
#     'Dosage': 5502/570,
#     'Reason': 5502/1245,
#     'Duration': 5502/122,
#     'ADE': 5502/4546
# }
# weight_dict

weight_dict = {
    'Drug': 1,
    'Strength': 1,
    'Form': 1,
    'Frequency': 1,
    'Route': 1,
    'Dosage': 1,
    'Reason': 5,
    'Duration': 1,
    'ADE': 5
}
weight_dict

{'Drug': 1,
 'Strength': 1,
 'Form': 1,
 'Frequency': 1,
 'Route': 1,
 'Dosage': 1,
 'Reason': 5,
 'Duration': 1,
 'ADE': 5}

In [ ]:
corpus.train[0]

Sentence[44]: "Since no new infection was found this was presumed [ * * 12-26 * * ] steroids and the leukocytosis improved with prednisone taper . WBC 12 on day of discharge . Hyperglycemia : Patient is not known to be a diabetic and was" → ["steroids"/Drug, "leukocytosis"/ADE, "prednisone"/Drug, "Hyperglycemia"/ADE]

In [ ]:
corpus.test[0]

Sentence[14]: "MEDICATIONS : Lipitor , Tylenol with Codeine , Dilantin , previously on Decadron q.i.d" → ["Lipitor"/Drug, "Tylenol with Codeine"/Drug, "Dilantin"/Drug, "Decadron"/Drug, "q.i.d"/Frequency]

In [7]:
# embedding_types = [
#     # word embeddings trained on PubMed and PMC
#     WordEmbeddings("pubmed"),
#     # flair embeddings trained on PubMed and PMC
#     FlairEmbeddings("pubmed-forward"),
#     FlairEmbeddings("pubmed-backward"),
# ]

# embeddings = StackedEmbeddings(embeddings=embedding_types)

# tagger = SequenceTagger(
#     hidden_size=256,
#     embeddings=embeddings,
#     tag_dictionary=tag_dictionary,
#     tag_type="ner",
#     rnn_type='LSTM',
#     tag_format='BIOES',
#     use_crf=True,
#     locked_dropout=0.5,
#     loss_weights=weight_dict
# )

tagger = SequenceTagger.load(f"{MODEL_PATH}/taggers/lstm-crf/final-model.pt")

2023-04-15 19:00:47,079 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


In [8]:
tagger.weight_dict = weight_dict
tagger.weight_dict

{'Drug': 1,
 'Strength': 1,
 'Form': 1,
 'Frequency': 1,
 'Route': 1,
 'Dosage': 1,
 'Reason': 5,
 'Duration': 1,
 'ADE': 5}

In [9]:
trainer = ModelTrainer(tagger, corpus)

trainer.train(
    base_path=f"{MODEL_PATH}/taggers/lstm-crf-augmented-weights-new",
    train_with_dev=False,
    max_epochs=20,
    patience=2,
    learning_rate=0.1,
    mini_batch_size=64,
    embeddings_storage_mode='none'
)

2023-04-15 19:00:47,800 ----------------------------------------------------------------------------------------------------
2023-04-15 19:00:47,802 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      '0-/root/.flair/embeddings/pubmed_pmc_wiki_sg_1M.gensim'
      (embedding): Embedding(1000001, 200)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4296, out_features=4296, bias=True)
  (rnn): LSTM(4296, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_fea

100%|██████████| 5/5 [00:05<00:00,  1.03s/it]

2023-04-15 19:01:47,569 Evaluating as a multi-label problem: False
2023-04-15 19:01:47,598 DEV : loss 0.04509250447154045 - f1-score (micro avg)  0.9357
2023-04-15 19:01:47,613 BAD EPOCHS (no improvement): 0
2023-04-15 19:01:47,620 saving best model


2023-04-15 19:01:53,356 ----------------------------------------------------------------------------------------------------
2023-04-15 19:02:06,524 epoch 2 - iter 4/44 - loss 0.09926120 - time (sec): 13.17 - samples/sec: 792.47 - lr: 0.100000
2023-04-15 19:02:12,873 epoch 2 - iter 8/44 - loss 0.09672658 - time (sec): 19.51 - samples/sec: 1060.73 - lr: 0.100000
2023-04-15 19:02:19,940 epoch 2 - iter 12/44 - loss 0.09761069 - time (sec): 26.58 - samples/sec: 1167.47 - lr: 0.100000
2023-04-15 19:02:24,016 epoch 2 - iter 16/44 - loss 0.09524747 - time (sec): 30.66 - samples/sec: 1347.14 - lr: 0.100000
2023-04-15 19:02:32,655 epoch 2 - iter 20/44 - loss 0.09410812 - time (sec): 39.30 - samples/sec: 1305.69 - lr: 0.100000
2023-04-15 19:02:36,995 epoch 2 - iter 24/44 - loss 0.09402572 - time (sec): 43.64 - samples/sec: 1399.63 - lr: 0.100000
2023-04-15 19:02:42,653 epoch 2 - iter 28/44 - loss 0.09147882 - time (sec): 49.30 - samples/sec: 1445.03 - lr: 0.100000
2023-04-15 19:02:47,213 epoch 2

100%|██████████| 5/5 [00:05<00:00,  1.13s/it]

...

2023-04-15 19:28:40,964 Evaluating as a multi-label problem: False
2023-04-15 19:28:41,012 DEV : loss 0.010277992114424706 - f1-score (micro avg)  0.9863
2023-04-15 19:28:41,032 BAD EPOCHS (no improvement): 1


2023-04-15 19:28:47,244 ----------------------------------------------------------------------------------------------------
2023-04-15 19:28:52,465 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


100%|██████████| 397/397 [05:53<00:00,  1.12it/s]


2023-04-15 19:34:47,602 Evaluating as a multi-label problem: False
2023-04-15 19:34:49,626 0.8939	0.9153	0.9044	0.8329
2023-04-15 19:34:49,633 
Results:
- F-score (micro) 0.9044
- F-score (macro) 0.8222
- Accuracy 0.8329

By class:
              precision    recall  f1-score   support

        Drug     0.8780    0.9490    0.9121     61167
    Strength     0.9356    0.9609    0.9481     42957
        Form     0.9187    0.9116    0.9152     41417
   Frequency     0.8572    0.8570    0.8571     36495
       Route     0.9606    0.9454    0.9530     30583
      Dosage     0.9112    0.9196    0.9154     23506
      Reason     0.8123    0.6964    0.7499      9533
         ADE     0.2553    0.6459    0.3660      1299
    Duration     0.8205    0.7497    0.7835      1982

   micro avg     0.8939    0.9153    0.9044    248939
   macro avg     0.8166    0.8484    0.8222    248939
weighted avg     0.8987    0.9153    0.9060    248939

2023-04-15 19:34:49,637 ---------------------------------------

{'test_score': 0.9044450972141729,
 'dev_score_history': [0.9357163447679447,
  0.94151212553495,
  0.9519977004886462,
  0.9677230046948357,
  0.962557603686636,
  0.9655969933506793,
  0.9673693329483107,
  0.9781149693609571,
  0.9798068481123793,
  0.9810329734461628,
  0.9799360279150916,
  0.9824766355140188,
  0.983941605839416,
  0.9853971962616821,
  0.9807692307692308,
  0.9868305531167691,
  0.9836448598130841,
  0.9848042080654588,
  0.9874158618671349,
  0.9862773722627737],
 'train_loss_history': [0.1150926335231143,
  0.0889642184590845,
  0.07726804988703964,
  0.06812617253133373,
  0.059881689637164524,
  0.052712130608283,
  0.04760506483867672,
  0.04024144389162618,
  0.03949490862415422,
  0.03678326583147904,
  0.035545039039439626,
  0.033865259297942873,
  0.03278104032798238,
  0.03068894996537396,
  0.030525389826756278,
  0.02914989172267381,
  0.028407666971300122,
  0.027240820906748103,
  0.025549457761909086,
  0.02517195056707234],
 'dev_loss_history': 

In [ ]:
# create example sentence
sentence = Sentence("Patients on 40 mg of Topelfate and Topoxy twice a day generally suffer from headache")

# token level predictions
tagger.predict(sentence, force_token_predictions=True)
print(sentence.to_tagged_string())

# predict tags and print
tagger.predict(sentence)
print(sentence.to_tagged_string())

Sentence[15]: "Patients on 40 mg of Topelfate and Topoxy twice a day generally suffer from headache" → ["40"/B-Strength, "mg"/E-Strength, "Topelfate"/S-Drug, "Topoxy"/S-Drug, "twice"/B-Frequency, "a"/I-Frequency, "day"/E-Frequency, "headache"/S-ADE]
Sentence[15]: "Patients on 40 mg of Topelfate and Topoxy twice a day generally suffer from headache" → ["40 mg"/Strength, "Topelfate"/Drug, "Topoxy"/Drug, "twice a day"/Frequency, "headache"/ADE]


In [ ]:
# create example sentence
sentence = Sentence("Patients on 40 mg of Topelfate and Topoxy for headache twice a day generally experience jdfadladkh")

# token level predictions
tagger.predict(sentence, force_token_predictions=True)
print(sentence.to_tagged_string())

# predict tags and print
tagger.predict(sentence)
print(sentence.to_tagged_string())

Sentence[16]: "Patients on 40 mg of Topelfate and Topoxy for headache twice a day generally experience jdfadladkh" → ["40"/B-Strength, "mg"/E-Strength, "Topelfate"/S-Drug, "Topoxy"/S-Drug, "headache"/S-Reason, "jdfadladkh"/S-ADE]
Sentence[16]: "Patients on 40 mg of Topelfate and Topoxy for headache twice a day generally experience jdfadladkh" → ["40 mg"/Strength, "Topelfate"/Drug, "Topoxy"/Drug, "headache"/Reason, "jdfadladkh"/ADE]
